## Importación de librerías

In [150]:
import matplotlib.pyplot as plt
import pandas as pd

from datetime import date

## Importación del dataset original

In [151]:
data_raw = pd.read_csv('../data/01-raw-properati.csv')
data_raw.head(3)

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,...,bathrooms,surface_total,surface_covered,currency,price_period,title,description,property_type,operation_type,price
0,556713,Propiedad,2019-11-29,9999-12-31,2019-11-29,-58.442399,-34.573623,Argentina,Capital Federal,Colegiales,...,2.0,NaN,NaN,USD,NaN,"Departamento en Venta en Belgrano, Capital fed...","Sup total por escritura: 96,47 m2 (cubiertos: ...",Departamento,Venta,259000.0
1,192912,Propiedad,2020-06-05,2020-06-08,2020-06-05,-58.430493,-34.606620,Argentina,Capital Federal,Almagro,...,2.0,77.0,67.0,USD,NaN,Departamento de 3 ambientes en Venta en Almagro,Excelente departamento de tres ambientes ampli...,Departamento,Venta,235500.0
2,238224,Propiedad,2020-07-01,9999-12-31,2020-07-01,-58.491760,-34.574123,Argentina,Capital Federal,Villa Urquiza,...,1.0,60.0,55.0,USD,NaN,Andonaegui 2600 4° - - Departamento en Venta,Excelente 3 ambientes al frente con balcón. Vi...,Departamento,Venta,175000.0


In [152]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 992192 entries, 0 to 992191
Data columns (total 25 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               992192 non-null  int64  
 1   ad_type          992192 non-null  object 
 2   start_date       992192 non-null  object 
 3   end_date         992192 non-null  object 
 4   created_on       992192 non-null  object 
 5   lat              838994 non-null  float64
 6   lon              839929 non-null  float64
 7   l1               992192 non-null  object 
 8   l2               992192 non-null  object 
 9   l3               934737 non-null  object 
 10  l4               225395 non-null  object 
 11  l5               4711 non-null    object 
 12  l6               0 non-null       float64
 13  rooms            503045 non-null  float64
 14  bedrooms         390699 non-null  float64
 15  bathrooms        773021 non-null  float64
 16  surface_total    448164 non-null  floa

In [153]:
data_raw.describe()

,id,lat,lon,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
count,992192.000000,838994.000000,839929.000000,0.0,503045.000000,390699.000000,773021.000000,448164.00000,4.369610e+05,9.478110e+05
mean,500338.438758,-59.266586,-34.793745,NaN,2.858577,2.086094,1.697637,493.51136,5.769533e+02,2.881687e+05
std,288685.497758,2.707383,3.066257,NaN,1.682454,2.705342,1.072722,4101.38179,7.383998e+04,5.560863e+06
min,1.000000,-180.000000,-54.842464,NaN,1.000000,-13.000000,1.000000,-200.00000,-1.700000e+02,0.000000e+00
25%,250259.750000,-58.756011,-34.921709,NaN,2.000000,1.000000,1.000000,50.00000,4.500000e+01,3.900000e+04
50%,500641.500000,-58.444383,-34.610508,NaN,3.000000,2.000000,1.000000,95.00000,7.500000e+01,9.800000e+04
75%,750507.250000,-58.058983,-34.459156,NaN,4.000000,3.000000,2.000000,270.00000,1.550000e+02,2.100000e+05
max,1000000.000000,-35.045025,85.051129,NaN,40.000000,900.000000,20.000000,200000.00000,2.700000e+07,3.100000e+09


In [154]:
data_raw.nunique()

id                 992192
ad_type                 1
start_date            346
end_date              450
created_on            346
lat                317046
lon                315745
l1                      4
l2                     42
l3                   1262
l4                    976
l5                     21
l6                      0
rooms                  37
bedrooms               83
bathrooms              20
surface_total        4821
surface_covered      2607
currency                4
price_period            3
title              454605
description        638785
property_type          10
operation_type          3
price               18833
dtype: int64

## Filtrado de variables

### Eliminación de columnas

Justificación de columnas borradas:

- Id: es único por registro
- ad_type: hay un solo tipo
- l6: es nulo para todos los registros
- title: no es útil para los modelos a implementar
- description: no es útil para los modelos a implementar

In [155]:
data_preprocessed = data_raw.drop(columns=["id", "ad_type", "l6", "title", "description"])

#### Filtrar columnas por determinados valores

En principio, decido guardar las columnas de latitud y longitud para evaluar su influencia en los modelos. Se me ocurre que pueden afectar el precio dentro de cada zona (CABA y GBA Norte, Oeste y Sur).

##### Cantidad de valores por columna

In [156]:
data_preprocessed["operation_type"].value_counts()

operation_type
Venta                762318
Alquiler             199399
Alquiler temporal     30475
Name: count, dtype: int64

In [157]:
data_preprocessed["currency"].value_counts()

currency
USD    722707
ARS    220109
UYU      2085
PEN        40
Name: count, dtype: int64

In [158]:
data_preprocessed["l1"].value_counts()

l1
Argentina         973422
Uruguay            17929
Estados Unidos       783
Brasil                58
Name: count, dtype: int64

In [159]:
data_preprocessed["l2"].value_counts()

l2
Capital Federal                 249738
Buenos Aires Costa Atlántica    178712
Bs.As. G.B.A. Zona Norte        127510
Bs.As. G.B.A. Zona Sur          112975
Santa Fe                         93111
Bs.As. G.B.A. Zona Oeste         73172
Córdoba                          60877
Buenos Aires Interior            22280
Mendoza                          11558
Neuquén                          10642
Maldonado                        10101
Montevideo                        6318
Río Negro                         5507
Tucumán                           5425
Entre Ríos                        5113
Misiones                          4981
Salta                             3085
San Luis                          2101
Chaco                             1145
Corrientes                        1052
San Juan                          1014
Chubut                             979
La Pampa                           910
Colonia                            750
Canelones                          607
Jujuy                 

##### Cantidad de valores por filtro

Siguiendo las recomendaciones:

- Voy a enfocarme en los valores de venta, ya que es el tipo de operation_type que tiene más registros, y asumo que los precios para los distintos tipos de operaciones (Venta, Alquiler y Alquiler temporal) se ven afectados por factores externos a los datos que tengo para trabajar.
- Para descartar problemas de convertibilidad de monedas, trabajaré solo con propiedades valuadas en ARS.
- Y simplemente por recomendación de la consigna, trabajaré con propiedades de CABA y GBA. 

In [160]:
data_preprocessed = data_preprocessed[
    (data_preprocessed["operation_type"] == "Venta")
    & (data_preprocessed["currency"] == "USD")
    & (data_preprocessed["l1"] == "Argentina")
    & (data_preprocessed["l2"].isin(["Capital Federal", "Bs.As. G.B.A. Zona Sur", "Bs.As. G.B.A. Zona Norte", "Bs.As. G.B.A. Zona Oeste"]))
]

data_preprocessed.shape[0]

394641

Respecto a los campos de fecha, asumo que:

- created_on: fecha de creación de la publicación de venta
- start_date: es la fecha de inicio de la publicación de venta
- end_date: es la fecha de fin de la publicación de venta, la cual NO garantiza que sea la fecha de venta

In [161]:
def calculate_date_differences(df, end_date_col='end_date', start_date_col='start_date', created_on_col='created_on'):

    df[created_on_col] = pd.to_datetime(df[created_on_col])
    df[start_date_col] = pd.to_datetime(df[start_date_col])
    df[end_date_col] = pd.to_datetime(df[end_date_col], errors="coerce")

    today = pd.Timestamp(date.today())

    time_delta_created_today = today - df[created_on_col]
    time_delta_start_today = today - df[start_date_col]
    time_delta_end_today = today - df[end_date_col]

    df['days_since_creation'] = time_delta_created_today.dt.days
    df['days_since_start'] = time_delta_start_today.dt.days
    df['days_since_end'] = time_delta_end_today.dt.days

    return df

data_preprocessed = calculate_date_differences(data_preprocessed)

C:\Users\alejo\AppData\Local\Temp\ipykernel_14580\1350242129.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[end_date_col] = pd.to_datetime(df[end_date_col], errors="coerce")


In [162]:
data_preprocessed[["days_since_end", "end_date"]].head(5)

,days_since_end,end_date
0,NaN,NaT
1,1977.0,2020-06-08
2,NaN,NaT
3,NaN,NaT
4,2259.0,2019-08-31


### Revisión de duplicados

#### Revisión inicial

Considerando todas las columnas (incluido el id), parece que no hay duplicados

In [163]:
data_raw.duplicated().value_counts()

False    992192
Name: count, dtype: int64

In [164]:
def get_duplicated_rows(df, columns_to_ignore):
    data_with_dropped_columns = df.drop(columns=columns_to_ignore)
    print(f"Columnas restantes: {data_with_dropped_columns.columns}")
    print("")
    print("Cantidad de columnas no repetidas (False) y repetidas (True)")
    return data_with_dropped_columns.duplicated().value_counts()

#### Suposiciones descartadas

Si asumiera que una misma propiedad puede publicarse varias veces, con distinto título, descripción, latitud y longitud

In [165]:
get_duplicated_rows(data_raw,["id","start_date", "end_date", "created_on", "title", "description", "lat", "lon"])

Columnas restantes: Index(['ad_type', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms',
       'bathrooms', 'surface_total', 'surface_covered', 'currency',
       'price_period', 'property_type', 'operation_type', 'price'],
      dtype='object')

Cantidad de columnas no repetidas (False) y repetidas (True)


False    567769
True     424423
Name: count, dtype: int64

Si asumiera que una misma propiedad puede publicarse varias veces, con distinta descripción, latitud y longitud

In [166]:
get_duplicated_rows(data_raw, ["id","start_date", "end_date", "created_on", "description", "lat", "lon"])

Columnas restantes: Index(['ad_type', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms',
       'bathrooms', 'surface_total', 'surface_covered', 'currency',
       'price_period', 'title', 'property_type', 'operation_type', 'price'],
      dtype='object')

Cantidad de columnas no repetidas (False) y repetidas (True)


False    673278
True     318914
Name: count, dtype: int64

Si asumiera que una misma propiedad puede publicarse varias veces, con distinto título y descripción.
Asumo que varias propiedades (por ejemplo, departamentos) pueden tener la misma latitud y longitud.

In [167]:
data_raw_without_apartments = data_raw[data_raw["property_type"].isin(["Departamento", "Otro", "PH", "Oficina"])]
get_duplicated_rows(data_raw_without_apartments, ["id", "start_date", "end_date", "created_on", "title", "description"])

Columnas restantes: Index(['ad_type', 'lat', 'lon', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms',
       'bedrooms', 'bathrooms', 'surface_total', 'surface_covered', 'currency',
       'price_period', 'property_type', 'operation_type', 'price'],
      dtype='object')

Cantidad de columnas no repetidas (False) y repetidas (True)


False    390034
True     287378
Name: count, dtype: int64

#### Suposiciones válidas

Si solo quito la columna id

In [168]:
get_duplicated_rows(data_raw_without_apartments, ["id"])

Columnas restantes: Index(['ad_type', 'start_date', 'end_date', 'created_on', 'lat', 'lon', 'l1',
       'l2', 'l3', 'l4', 'l5', 'l6', 'rooms', 'bedrooms', 'bathrooms',
       'surface_total', 'surface_covered', 'currency', 'price_period', 'title',
       'description', 'property_type', 'operation_type', 'price'],
      dtype='object')

Cantidad de columnas no repetidas (False) y repetidas (True)


False    656114
True      21298
Name: count, dtype: int64

Si asumiera que una misma propiedad puede publicarse varias veces, con distinta descripción.
Asumo que varias propiedades (por ejemplo, departamentos) pueden tener la misma latitud y longitud.

In [169]:
get_duplicated_rows(data_raw_without_apartments, ["id", "start_date", "end_date", "created_on", "description"])

Columnas restantes: Index(['ad_type', 'lat', 'lon', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'rooms',
       'bedrooms', 'bathrooms', 'surface_total', 'surface_covered', 'currency',
       'price_period', 'title', 'property_type', 'operation_type', 'price'],
      dtype='object')

Cantidad de columnas no repetidas (False) y repetidas (True)


False    422573
True     254839
Name: count, dtype: int64

### Imputación de datos

Imputaré la media de la longitud de la zona en aquellos registros donde la latitud tenga valor y la longitud sea nula

In [170]:
data_preprocessed[["lat","lon"]].isna().value_counts()

lat    lon  
False  False    331477
True   True      62718
       False       446
Name: count, dtype: int64

In [174]:
data_preprocessed[(data_preprocessed["lat"].isna()) & (data_preprocessed["lon"].notna())].shape[0]

446

In [ ]:
lat_mean_by_zone = data_preprocessed.groupby("l2")["lat"].transform("mean")
condition_for_filling = data_preprocessed["lat"].isna() & data_preprocessed["lon"].notna()

data_preprocessed["lat"] = data_preprocessed["lat"].mask(
    condition_for_filling,
    lat_mean_by_zone
)

lat
False    331923
True      62718
Name: count, dtype: int64

## Exploración de variables relevantes
    *   Superficie (`surface_total`, `surface_covered`)
    *   Ambientes, dormitorios, baños
    *   Ubicación (`place_name`, `state_name`, `lat`, `lon`)
    *   Tipo de propiedad (`property_type`)
    *   Precio (`price_usd`)

In [ ]:
# TODO: Revisar variación de precios a medida pasa el tiempo

## Limpieza y tratamiento de datos
    *   Gestionar valores duplicados y nulos irrelevantes.
    *   Decidir qué variables conservar para el modelo.
    *   Imputar valores faltantes (si corresponde).
    *   Detectar y manejar outliers (ej. establecer límites razonables de superficie/precio).

## Generar un dataset limpio final y guardarlo como `data/processed.csv`.